In [8]:
from langchain.chains import LLMMathChain
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

In [15]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, base_url="", temperature=0, model="gpt-3.5-turbo-0613")
search = SerpAPIWrapper(serpapi_api_key="")

In [16]:
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
db = SQLDatabase.from_uri("sqlite:///./Chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
    Tool(
        name="FooBar-DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context",
    ),
]

In [17]:
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [18]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory=memory,
)

In [19]:
agent.run("hi")



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


'Hello! How can I assist you today?'

In [20]:
agent.run("my name is bob")



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I help you today?

> Finished chain.


'Nice to meet you, Bob! How can I help you today?'

In [24]:
agent.run("whats my name")



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.


'Your name is Bob.'